In [36]:
## library
import random
import numpy as np
import torch
from torch.autograd.variable import Variable
from torch import nn, optim
import visdom
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import sys
import numpy as np
from keras.utils.vis_utils import plot_classifier
import time
from tqdm import tqdm
from torch.utils.data import Subset
##Classifier pytorch    

In [6]:
noise_dim=100
batch_size=64
num_img_channel=1
img_size=28
sample_interval=64          



In [19]:
class Mnist_Classifier(nn.Module):
    def __init__(self):
        super(Mnist_Classifier,self).__init__()

    
        self.fc_block=nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28,128,bias=False),
            nn.BatchNorm1d(128),
            nn.Tanh(),
            nn.Linear(128,10)
        )
            
    def forward(self,img):
        l2_value=self.fc_block(img)
        l2_value=l2_value.unsqueeze_(dim=2).unsqueeze_(dim=3)
        return l2_value
    


In [161]:
input_shape=[1,28,28]


tensor([4, 7, 8,  ..., 5, 6, 8])

In [29]:
classifier=Mnist_Classifier()
classifier.cuda()
classifier

Mnist_Classifier(
  (fc_block): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=False)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Tanh()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [30]:
#Dataset Mnist:
def mnist_data():
    compose = transforms.Compose(
        [transforms.Resize(img_size),
         transforms.ToTensor(),
         transforms.Normalize((0.5,), (0.5,)),  # Remarquez que les paramètres sont des tuples à un élément seulement
         transforms.Grayscale()
         ])
    output_dir = './data/mnist'
    return datasets.MNIST(root=output_dir, train=True,
                          transform=compose, download=True)


mnist = mnist_data()
train_indices = range(0, int(0.8 * len(mnist)))  # Utilisez les 80% premières images pour l'entraînement
train_dataset = Subset(mnist, train_indices)



batch_iterator = DataLoader(train_dataset, shuffle=True, batch_size=batch_size) # List, NCHW format.


cuda = torch.cuda.is_available()
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
test_indices = range(int(0.8 * len(mnist)), len(mnist))
test_dataset = Subset(mnist, test_indices)




In [31]:
display_server="http://localhost"
display_port=8097
try:
    
    vis = visdom.Visdom(server=display_server, port=display_port, raise_exceptions=True) # Create vis env.
except ImportError:
    vis = None
else:
    vis.close(None) # Clear all figures.



Setting up a new session...


In [152]:
K=random.randint(1,12000)
vis.images(mnist.data[48000+K,:,:])
images=mnist.data[test_indices,:,:].unsqueeze(1).cuda().float()
classifier(images)[K,:,:,:].argmax()





tensor(9, device='cuda:0')

In [68]:
lr=0.0001
b1=0.5
b2=0.999
optimizer=optim.Adam(classifier.parameters(),lr=lr,betas=(b1,b2))
loss_C=nn.CrossEntropyLoss()
seed = Variable(Tensor(25, noise_dim, 1, 1).normal_(0, 1), requires_grad=False) # To track the progress of the GAN.


In [46]:
from tqdm import tqdm

def train(n_epochs):
    for epoch in tqdm(range(n_epochs)):
        print('Epoch {}'.format(epoch))
        for i, (batch, labels) in enumerate(batch_iterator):
            
            imgs = Variable(batch.type(Tensor), requires_grad=False)
            
        
            labels = Variable(labels.type(Tensor), requires_grad=False)
            optimizer.zero_grad()

            output = classifier(imgs)

            loss = loss_C(output, labels.unsqueeze(1).unsqueeze(2).long() )

            loss.backward()
            optimizer.step()


In [212]:
train(6)

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 0


 17%|█▋        | 1/6 [00:11<00:59, 11.89s/it]

Epoch 1


 33%|███▎      | 2/6 [00:23<00:46, 11.58s/it]

...

 67%|██████▋   | 4/6 [00:46<00:23, 11.53s/it]

Epoch 4


 83%|████████▎ | 5/6 [00:57<00:11, 11.47s/it]

Epoch 5


100%|██████████| 6/6 [01:09<00:00, 11.51s/it]


In [206]:
K=random.randint(1,12000)
vis.images(mnist.data[48000+K,:,:])
images=mnist.data[test_indices,:,:].unsqueeze(1).cuda().float()
classifier(images)[K,:,:,:].argmax()




tensor(2, device='cuda:0')

In [213]:
labels=mnist.targets[48000:].unsqueeze(1).unsqueeze(2).cuda()   
predicted=torch.argmax(classifier(images),dim=1)
(torch.sum(labels==predicted)*100/12000)





tensor(82.3833, device='cuda:0')